In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab10.ipynb")

# Lab 10: Elliptic Curve Cryptography
Contributions From: Ryan Cottone

Welcome to Lab 10! In this lab, we will explore the properties of elliptic curves and their applications to cryptography.

### Helpers

In [ ]:
import random

In [ ]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modularInverse(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

def findSquareRoot(N,p):
    N = N % p
    
    if pow(N,(p-1)//2,p)==1:
        if (p%4) == 3:
            x1= pow(N,(p+1)//4, p)
            y1= p - x1
            return (x1,y1)
        else:
            for x in range(1,((p-1)//2)+1):
                if N == pow(x, 2, p):
                    return (x,p-x)
    return []

def evaluateP(x, E, p):
    evaled = ((pow(x, 3, p) + E[0]*x + E[1])%p)
    
    if evaled == 0:
        return [0]
    
    return findSquareRoot(evaled, p)

def pointOnCurve(P,E,p):
    assert isElliptic(E, p), "Invalid elliptic curve (has zero discriminant)"
    
    if P[0] == 'O':
        return True
    
    evaled = evaluateP(P[0], E, p)
    
    if len(evaled) == 0:
        return False
    
    return (evaled[0]%p == P[1]) or (evaled[1]%p == P[1])

# Elliptic Curves

An **elliptic curve** is a curve of the form $y^2 = x^3 + ax + b$. They looks something like this:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/ECClines-3.svg/2560px-ECClines-3.svg.png" style="width:800px">

A crucial property of elliptic curves for cryptographic purposes is having a **nonzero discriminant**, which just means we require $$4a^3 + 27b^2 \neq 0$$

What happens with a zero determinant? Some points on the curve will be non-singular, which roughly means there exists and "overlap" of two parts of the curve at one point, like this:

<img src="./bad_elliptic_curve.png" style="width:400px">

We first need a way to check for these "bad curves". Note that we do elliptic curve operations $\mod p$ for some prime p, so you need to reduce mod p.

**Question 1**: Implement isElliptic!

In [ ]:
def isElliptic(E, p):
    """
    Takes in curve E as [A, B] and modulo p. 
    Returns whether the discriminant of E is nonzero.
    """
    
    # Make sure to do this modulo p.
    discriminant = ...
    
    return discriminant != 0 

In [ ]:
grader.check("q1")

## Elliptic Curve Operations

Elliptic curves seem interesting on the surface, but how do we actually use them? First, we need to define the most basic operation: *addition*.

To "add" two points on an elliptic curve (denoted $P \oplus Q$ for points P,Q), we draw a line through them and mark where the line intersects the curve at a third location (denoted -R). We then reflect -R across the x-axis to get our final point R. For example:
<img src="./elliptic_curve_add_example.png" style="width:400px">

Seems simple enough, but there are a few edge cases we need to handle:

**First Edge Case: Self-Addition**

This isn't so much of an edge case as a fundamental property, but let's call it one anyway. To add a point to itself, we take the **tangent line** at P and see where it intersects, then proceed as before. For example:

<img src="./self_addition_elliptic.png" style="width:400px">

**Second Edge Case: Vertical Addition**

What happens when we want to add P and -P (for example, (0,2) and (0,-2)). We draw a line between them, but it's totally vertical and never intersects the curve again. For these cases, we define a **point at infinity** as $O$. For example:

<img src="./add_vertical.png" style="width:400px">

This line never intersects a third point, so $P \oplus Q = O$. Crucially $P \oplus O = P$ for all $P$. You can visualize this as a vertical line from $P$ intersects at $-P$, which is then reflected back to $P$:

<img src="./addition_point_at_infty.png" style="width:400px">

### Closed-Form Formulas 

Hopefully addition makes sense geometrically, but how do we express these operations in terms of formuals? Thankfully, there exist nice closed-form formulas for addition (derivation of which is out of scope). Given points $P = (x_1, y_1)$ and $Q = (x_2, y_3)$ we first find $\lambda$ as the following:

If $P \neq Q$: 
$$ \lambda = \frac{y_2 - y_1}{x_2 - x_1}$$

Crucially, since we are working $\mod p$, we have to use modular inverses instead of division:
$$\lambda = (y_2 - y_1)(x_2 - x_1)^{-1} \mod p$$

where $^{-1}$ denotes the modular inverse over $\mod p$.

If $P = Q$ (self-addition): 
$$ \lambda = \frac{3x_1^2 + A}{2y_1} $$
Over modular arithmetic: 
$$ \lambda = (3x_1^2 + A)(2y_1)^{-1} \mod p $$



Regardless of $P, Q$ we then find:

$$x_3 = \lambda^2 - x_1 - x_2\\
y_3 = \lambda(x_1 - x_3) - y_1$$

Our new point $P \oplus Q$ is therefore $(x_3, y_3)$.

**Question 2:** Implement addPoints! Remember to use the modular arithmetic version of the formula.

In [ ]:
def addPoints(P,Q,E,p):
    """
    Given points P,Q of the form (x,y), curve E of the form [A,B], and prime p, find P + Q.
    """
    
    assert isElliptic(E, p), "Invalid elliptic curve (has zero discriminant)"
    assert pointOnCurve(P, E, p), "Point P is not on the elliptic curve"
    assert pointOnCurve(Q, E, p), "Point Q is not on the elliptic curve"

    multi = 1 # Lambda

    if P == 'O': # If P is the point at infinity
        ...
    elif Q == 'O': # If Q is the point at infinity
        ...
    elif P[0] == Q[0] and P[1] == (-Q[1] % p): # If P and Q are on a vertical line (share the same x-coord)
        ...
    elif P[0] == Q[0]: # If P = Q
        multi = ((3*pow(P[0], 2, p) + E[0])*modularInverse(2*P[1], p)) % p # Set lambda
    else: 
        multi = ((Q[1] - P[1])*modularInverse((Q[0] - P[0])%p, p))%p # Set lambda

    x_3 = ...
    y_3 = ...

    return (x_3, y_3)

In [ ]:
grader.check("q2")

## Elliptic Curve Multiplication

The fundamental definition of multiplication is repeated addition -- and the same applies to elliptic curves. We define scalar multiplication of a point as adding the point to itself that many times. For example, $3P = P \oplus P \oplus P$. This is roughly analogous to *modular exponentiation*, a surprising fact we will explore a little later on in the lab.


### The Double and Add Algorithm

Much like modular exponentiation, naively calculating $nP$ for very large $n$ can be inefficient. Thankfully, there exists a very fast algorithm for computing $nP$ named the double-and-add algorithm. It relies on the fact that for any $a,b$, $(a+b)P = aP \oplus bP$.

Given some integer $n$, we can split it up into additive factors and add those points instead. If we do this for powers of 2, it becomes very efficient! Consider the fact that $aP \oplus aP = (a+a)P = (2A)P$, meaning we can double the coefficient of a point in constant time. This means computing the $2^iP$ for integer $i$ is quite fast. 

Consider the binary representation of $n$, for example, 19 = 10011. This means $19 = 2^4 + 2^1 + 2^0$, and $19P = 2^4P \oplus 2^1P \oplus 2^0P$. To calculate these powers of $2$, we can use the formula:

$$2^iP = (2^{i-1} P \oplus 2^{i-1} P) = 2(2^{i-1} P)$$

and build up from $2^0 P = P$.

Our algorithm therefore:

1. Finds the binary representation of $n$, either on-the-fly or beforehand
2. Sums $2^iP$ where the i-th bit from the right equals 1

**Question 3:** Implement the double-and-add algorithm!

In [ ]:
def doubleAndAdd(P,n,E,p):
    assert isElliptic(E, p), "Invalid elliptic curve (has zero discriminant)"
    assert pointOnCurve(P, E, p), "Point P is not on the elliptic curve"
    
    point = P
    finalPoint = 'O'
        
    # What this while loop does is iteratively find the binary representation
    # starting from the least significant bit. It is faster than finding the representation first.
    while n > 0:
        r = n % 2
        n //= 2
        
        if r == 1: # Checks if this binary bit is 1
            # If the bit is one, add the current power of two point to the finalPoint
            finalPoint = ...
        
        # Double the current power of two point to the next power of two
        point = ...
    
    return finalPoint

In [ ]:
grader.check("q3")

## Elliptic Curve Diffie Hellman

Now to showcase perhaps one of the two most useful applications of elliptic curves: Diffie-Hellman! We covered
the Diffie-Hellman key exchange before using modular arithmetic, but using the elliptic-curve version provides more 
security with less key bits. This is why ECDH is used over modular-arithmetic based DH in most modern systems. 

As a quick refresher, Diffie-Hellman involves two parties (Alice and Bob) trying to derive a shared secret without an eavesdropper (Eve) 
figuring out the secret. Alice and Bob share the public parameters of $(g, N)$ respectively, where $g$ is a generator over $\mod N$. 

Alice and Bob then generate the secrets $a$ and $b$ respectively. They find $g^a \mod N$ and $g^b \mod N$ using them, and share both
aforementioned values. Once received, Alice finds $(g^b)^a \equiv g^{ab} \mod N$ and Bob finds $(g^a)^b \equiv g^{ab} \mod N$, which is 
their new shared secret.

Recall our parallels from earlier -- multiplication in elliptic curves is roughly equal to exponentiation in 
modular arithmetic. So, $aP$ for a generator point $P$ over $\mod N$ can be thought of as similar to finding $g^a \mod N$. (A generator 
point is a point that generates many unique points before wrapping around to itself) Crucially, scalar multiplication of 
elliptic curves is associative, so $b(aP) = a(bP) = (ab)P$. With that, we can identify the ECDH cryptosystem:

1. Alice and Bob share a generator point $P$ and the modulus $N$.
2. {Alice generates $a \in [2, n-1]$ where $n = \text{ord(}P\text{)}$ (This part is not too important to understand, just think of order as the number of unique points we can generate starting at P. We don't want a scalar value larger than that, or it will wrap around mod the order of P)
3. Bob does the same as Alice in 2) for a different $b$.
4. Alice finds $aP$, and Bob finds $bP$ using the double-and-add algorithm (or similarly efficient algorithm)
5. Alice and Bob both send $aP$ and $bP$ over the insecure channel. 
6. Alice receives $bP$ and computes $a(bP) = (ab)P$. 
7. Bob receives $aP$ and computes $b(aP) = (ab)P$.

Since Eve only has $aP$ and $bP$, she can only find $aP + bP = (a+b)P$, not 
$(ab)P$. ECDH is extremely similar to its modular arithmetic counterpart! The 
computation problem of finding $(ab)P$ from $aP$ and $bP$ is known as the 
**Elliptic Curve Diffie-Hellman Problem (ECDHP)**, and is believed to 
be computationally hard.


**Question 4**: Implement Elliptic Curve Diffie-Hellman!

In [ ]:
def generateECDH(generatorPoint, E, p, N):
    """
    Given a generator point P, curve E, prime p, and bound N, returns nP for some random integer n in [0, N-1].
    Returns (nP, n)
    """
    
    n = random.randrange(2, N) # [2, N-1]
    
    nP = ...
    
    return nP, n

In [ ]:
def combineECDH(publicPoint, k, E, p):
    """
    Takes in point publicPoint, secret k, the curve E, and prime p. 
    publicPoint can be either Bob's public or Alice's public key (aP, bP),
    and k is the opposite person's private key (n).
    Returns the shared secret (ab)P.
    """
    
    ...

Let's work through an example of ECDH! We will be using the real-world curve **Secp256k1** defined as $y^2 = x^3 + 7$.

In [ ]:
curve = [0,7]
prime = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
generatorPoint = (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
subgroupOrder = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

Alice is the first to generate her public/private keypair using our `generateECDH` function.

In [ ]:
alice_public, alice_private = generateECDH(generatorPoint, curve, prime, subgroupOrder)

# Make sure we generated the public key as priv_key*generatorPoint!
assert alice_public == doubleAndAdd(generatorPoint, alice_private, curve, prime)

Next, Bob generates his public/private keypair using the same methods.

In [ ]:
bob_public, bob_private = generateECDH(generatorPoint, curve, prime, subgroupOrder)

# Make sure we generated the public key as priv_key*generatorPoint!
assert bob_public == doubleAndAdd(generatorPoint, bob_private, curve, prime)

They then send their public messages over an insecure channel. Alice then takes `bob_public` and recovers the shared secret using `combinedECDH`.

In [ ]:
alice_shared_secret = combineECDH(bob_public, alice_private, curve, prime)

Bob does the same with Alice's public message.

In [ ]:
bob_shared_secret = combineECDH(alice_public, bob_private, curve, prime)

We verify that they derived the same secret:

In [ ]:
print("Alice secret:", (hex(alice_shared_secret[0]), hex(alice_shared_secret[1])))
print("Bob secret:", (hex(bob_shared_secret[0]), hex(bob_shared_secret[1])))

if alice_shared_secret == bob_shared_secret:
    print('\nSecrets match!')
else:
    print('\nSomething went wrong, check your functions again!')

In [ ]:
grader.check("q4")

Congrats on finishing Lab 10!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)